In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')


Processing-part-1: extracting raw table

In [3]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue

    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

part-2: integrating Postal codes with more than 1 neighbour

In [6]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 78


creating an appropriate DF

In [8]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M3K,North York,CFB Toronto
1,M2J,North York,Henry Farm
2,M5R,Central Toronto,"The Annex, Yorkville"
3,M6H,West Toronto,Dovercourt Village
4,M1G,Scarborough,Woburn
5,M7A,Downtown Toronto,Queen's Park
6,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
7,M1S,Scarborough,Agincourt
8,M4H,East York,Thorncliffe Park
9,M4B,East York,"Woodbine Gardens, Parkview Hill"


In [10]:
df_toronto.shape

(78, 3)

In [12]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coordenadas = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [14]:
print(coordenadas.shape)
coordenadas.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
toronto_df = df_toronto.set_index('Postcode')
coordenadas_df = coordenadas.set_index('Postal Code')
toronto_final = pd.concat([toronto_df, coordenadas_df], axis=1, join='inner')

toronto_final.index.name = 'Postcode'
toronto_final.reset_index(inplace=True)

print(toronto_final.shape)
toronto_final.head()

(78, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3K,North York,CFB Toronto,43.737473,-79.464763
1,M2J,North York,Henry Farm,43.778517,-79.346556
2,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678
3,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
4,M1G,Scarborough,Woburn,43.770992,-79.216917


Part 3: Explore the neighborhoods in Toronto

In [18]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes
import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jeison/anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           2 KB  conda-forge
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.8.2                |           py37_0         3.0 MB  conda-forge
    conda-package-handling-1.6.0|   py37h516909a_1         942 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-6

In [19]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.653963, -79.387207.


create a Map of Toronto - Using Folium 

In [22]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Postcode'], toronto_final['Borough'], toronto_final['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Reduce the number of Boroughs to explore To reduce the numbers of calls to FourSquare API, we will only explore boroughs that have Toronto in their names

In [24]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_central_df = toronto_final[toronto_final['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print(toronto_central_df.shape)
toronto_central_df.head()

(28, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678
1,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
3,M6K,West Toronto,"Exhibition Place, Parkdale Village",43.636847,-79.428191
4,M4T,Central Toronto,Moore Park,43.689574,-79.383160


In [27]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['Postcode'], toronto_central_df['Borough'], toronto_central_df['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [29]:
CLIENT_ID = 'OFW3KNIFXUC5QP3FBCHJUQAQHTMCIW012MF4XTMQZ3JI40YY'
CLIENT_SECRET = 'KHCOQXBMX54WX52CUAY3ZPEBY4E005MYBFIGJEN2AJEJNJVV' # your Foursquare Secret
VERSION = '20190330' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OFW3KNIFXUC5QP3FBCHJUQAQHTMCIW012MF4XTMQZ3JI40YY
CLIENT_SECRET:KHCOQXBMX54WX52CUAY3ZPEBY4E005MYBFIGJEN2AJEJNJVV


In [30]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['Postcode'], toronto_central_df['Borough'], toronto_central_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [31]:

venues_df = pd.DataFrame(venues)
venues_df.columns = ['Postcode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()


(768, 9)


,Postcode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5R,Central Toronto,"The Annex, Yorkville",43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
1,M5R,Central Toronto,"The Annex, Yorkville",43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,M5R,Central Toronto,"The Annex, Yorkville",43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
3,M5R,Central Toronto,"The Annex, Yorkville",43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant
4,M5R,Central Toronto,"The Annex, Yorkville",43.67271,-79.405678,Fet Zun,43.675147,-79.406346,Middle Eastern Restaurant


In [34]:
venues_df.groupby(['Postcode', 'Borough', 'Neighbourhood'])['VenueName'].count()

Postcode  Borough           Neighbourhood                                           
M4K       East Toronto      Riverdale                                                    41
M4N       Central Toronto   Lawrence Park                                                 4
M4T       Central Toronto   Moore Park                                                    2
M4W       Downtown Toronto  Rosedale                                                      4
M4Y       Downtown Toronto  Church and Wellesley                                         83
M5A       Downtown Toronto  Harbourfront                                                 47
M5E       Downtown Toronto  Berczy Park                                                  56
M5K       Downtown Toronto  Design Exchange, Toronto Dominion Centre                    100
M5P       Central Toronto   Forest Hill North                                             4
M5R       Central Toronto   The Annex, Yorkville                                       

In [35]:
len(venues_df['VenueCategory'].unique())

182

Analyze venues in each area

In [36]:
# one hot encoding
toronto_central_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_central_onehot['Postcode'] = venues_df['Postcode'] 
toronto_central_onehot['Borough'] = venues_df['Borough'] 
toronto_central_onehot['Neighbourhood'] = venues_df['Neighbourhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_central_onehot.columns[-3:]) + list(toronto_central_onehot.columns[:-3])
toronto_central_onehot = toronto_central_onehot[fixed_columns]

print(toronto_central_onehot.shape)
toronto_central_onehot.head()

(768, 185)


,Postcode,Borough,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M5R,Central Toronto,"The Annex, Yorkville",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5R,Central Toronto,"The Annex, Yorkville",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5R,Central Toronto,"The Annex, Yorkville",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5R,Central Toronto,"The Annex, Yorkville",0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,M5R,Central Toronto,"The Annex, Yorkville",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Get the frequency of occurance of each category in an area

In [38]:
toronto_central_venues_freq = toronto_central_onehot.groupby(['Postcode', 'Borough', 'Neighbourhood']).mean().reset_index()
print(toronto_central_venues_freq.shape)
toronto_central_venues_freq.head()

(19, 185)


,Postcode,Borough,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4K,East Toronto,Riverdale,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.02439,0.0,0.0,0.0,0.000000,0.0,0.000000,0.024390
1,M4N,Central Toronto,Lawrence Park,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,M4T,Central Toronto,Moore Park,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,M4W,Downtown Toronto,Rosedale,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.25000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,M4Y,Downtown Toronto,Church and Wellesley,0.012048,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012048,0.012048,0.00000,0.0,0.0,0.0,0.012048,0.0,0.012048,0.012048


Get 10 most occurance venue types in each area